In [39]:
using Plots, Colors, DataFrames, CSV, GLM, Statistics

cd("/Users/junwong/Dropbox/Second Year/Dingel - Trade/Assignments")

# Calibrating sector level data

In [18]:
df = DataFrame(CSV.File("data/bilateral_trade_country/bilateral_trade_sector_country.csv"))

# Total destination sector; collapse to country-origin/dest + sector-origin level
bilateral_spending_by_sectororg = combine(groupby(df, [:country_org, :country_dest, :sector_org]), :trade_flow2014 => sum, renamecols=false);
rename!(bilateral_spending_by_sectororg, :trade_flow2014 => :trade_flow2014_sectororg)

# Total spending by sector in country n 
spending_by_sectororg = combine(groupby(bilateral_spending_by_sectororg, [:country_dest, :sector_org]), :trade_flow2014_sectororg => sum, renamecols=false);
rename!(spending_by_sectororg, :trade_flow2014_sectororg => :spending_sectororg)
bilateral_spending_by_sectororg = leftjoin(bilateral_spending_by_sectororg, spending_by_sectororg, on=[:country_dest, :sector_org])
transform!(bilateral_spending_by_sectororg, [:trade_flow2014_sectororg, :spending_sectororg] => (./) => :pi_ni_j);

df = leftjoin(df, bilateral_spending_by_sectororg, on=[:country_org, :country_dest, :sector_org]);


Compute $\gamma$

In [40]:
# Total revenue of sector k in country n 
bilateral_revenue_by_sectordest = combine(groupby(df, [:country_org, :country_dest, :sector_dest]), :trade_flow2014 => sum, renamecols=false);
rename!(bilateral_revenue_by_sectordest, :trade_flow2014 => :trade_flow2014_sectordest)
revenue_by_sectordest = combine(groupby(bilateral_revenue_by_sectordest, [:country_org, :sector_dest]), :trade_flow2014_sectordest => sum, renamecols=false)
rename!(revenue_by_sectordest, [:country_org, :trade_flow2014_sectordest] .=> [:country, :revenue_sectordest])

cross_sector_spending = combine(groupby(df, [:country_dest, :sector_dest, :sector_org]), :trade_flow2014 => sum, renamecols=false)
rename!(cross_sector_spending, [:country_dest, :trade_flow2014] .=> [:country, :spending_cross_sector])

cross_sector_spending = leftjoin(cross_sector_spending, revenue_by_sectordest, on=[:country, :sector_dest])
transform!(cross_sector_spending, [:spending_cross_sector, :revenue_sectordest] => (./) => :gamma_n_jk)

# Average gamma across countries 
gamma_bar = combine(groupby(cross_sector_spending, [:sector_dest, :sector_org]), :gamma_n_jk => mean)

,sector_dest,sector_org,gamma_n_jk_mean
,Int64,Int64,Float64
1,0,1,0.0203131
2,0,2,0.00145403
3,0,3,0.00194419
4,0,4,0.00447138
5,0,5,0.0575723
6,0,6,0.0166605
7,0,7,0.00137359
8,0,8,0.00195332
9,0,9,0.000971318
